In [12]:
#Cài đặt thư viện 
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 14})
import numpy as np 
import pandas as pd 
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error
from pmdarima.arima import auto_arima
import statsmodels.api as sm
from IPython.display import display, Markdown

In [13]:
#Tiền xử lý dữ liệu
def data_preprocessing(data_src):
  df = pd.read_csv(data_src, parse_dates=True,
                       index_col=0).sort_values(by='Date', ascending=True)
  #Xóa dấu , và chuyển về float
  df['Price']=df['Price'].replace(',','',regex=True).astype(float)
  
  df = df[['Price']]
  return df

In [21]:
#Lấy số lượng dữ liệu của tập train, test, validation
def get_split_size(df1, train_ratio, test_ratio):
    train_size = int(train_ratio * len(df1))
    test_size = int(test_ratio * len(df1))
    val_size = len(df1) - train_size - test_size
    return train_size, test_size, val_size

In [14]:
#Chia dữ liệu
def split_data(df1,train_size, test_size, val_size):
  train_data = df1[:train_size]
  test_data = df1[train_size:train_size+test_size]
  val_data = df1[train_size+test_size:]
  return train_data, test_data, val_data

In [15]:
def visualize(label ,pred, model_name):
  plt.figure(figsize=(16,9))
  plt.plot(label, label = 'history')
  plt.plot(pred, label='predict')
  plt.title(model_name)
  plt.xlabel('Date')
  plt.ylabel('Revenues')
  plt.legend()
  plt.show()

In [17]:
#Làm hàm cho vào mảng seasonal_arr và chạy qua với auto arima để tìm ra chu kỳ tốt nhất
#Đối với mỗi lần lặp ta sẽ xét xem model nào có chỉ số AIC tốt nhất để lấy
def grid_search(arr,train_df,exog_train):
    best_aic=0
    for i in arr:
        print("Seasonal Period: ",i)
        model=auto_arima(train_df,exog=exog_train,seasonal=True,m=i, error_action='ignore', suppress_warnings=True)
        print("AIC: ",model.aic())
        print("------------------")
        if model.aic()<best_aic or best_aic==0:
            best_aic=model.aic()
            best_model=model
    return best_model

In [19]:
#Hàm xây dựng model bằng SARIMAX với statmodels
#Lý do không dùng auto arima là vì auto arima tự động chọn ra model tốt nhất
#nên biến ngoại sinh sẽ không được sử dụng và tự động bỏ ra
#Vì vậy ta sẽ xây dựng model bằng thư viện statmodels
#để có thể ép model sử dụng biến ngoại sinh
def build_model(train_df,exog_train,SEASONAL_ARR):
    search=grid_search(SEASONAL_ARR,train_df,exog_train)
    p,d,q=search.order
    P,D,Q,m=search.seasonal_order
    model_fit=sm.tsa.statespace.SARIMAX(endog=train_df,exog=exog_train, 
                                        order=(p, d, q), 
                                        seasonal_order=(P,D,Q,m))
    model=model_fit.fit()
    print(model.summary())
    return model

In [20]:
#Hàm dự đoán trên tập test và tập validation
def predict_test_and_val(model,train_size,test_size,val_size,test_df,val_df,exog_test,exog_val):
    #Quá trình test
    y_test=np.array(test_df)
    y_pred=model.predict(start=train_size,end=train_size+test_size-1,exog=exog_test)

    #Quá trình validate
    y_val=np.array(val_df)
    y_pred_val=model.predict(start=train_size,end=train_size+val_size-1,exog=exog_val)
    return y_pred,y_pred_val

In [ ]:
#Visualization
def visualize_overview(train_df,test_df, val_df, test_predict, val_predict, model_name):
  plt.figure(figsize=(12,6))
  plt.plot(train_df)
  plt.plot(test_df)
  plt.plot(test_df.index,test_predict)
  plt.plot(val_df)
  plt.plot(val_df.index, val_predict)
  #plt.plot(y_next_30_days)
  plt.title(model_name)
  plt.xlabel('Ngày')
  plt.ylabel('Giá cuối trong ngày')
  #plt.legend(['Train', 'Test', 'Predictions','Validate','Val','Next 30 days'], loc='lower right')
  plt.legend(['Train','Test','Predict','Validate','ValidatePred'], loc='lower right')
  plt.show()

In [1]:
def mda_cal(actual: np.ndarray, predicted: np.ndarray):
    actual = actual.values
    predicted = predicted.values
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - actual[:-1])).astype(int))

In [ ]:
#Các chỉ số Đánh giá 
def deviation(test, pred):
  rmse=np.sqrt(mean_squared_error(test,pred))
  print(f"RMSE: {rmse:.2f}")
  mape=mean_absolute_percentage_error(test,pred)
  print(f"MAPE: {mape*100:.2f}%")
  mda = mda_cal(test, pred)
  print(f"MDA: {mda*100:.2f}")

In [ ]:
#Làm hàm tổng quát cho vào 1 ngân hàng và 1 tỉ lệ chia tập train/test/validation
#Hàm này sẽ chạy model SARIMAX với ngân hàng và tỉ lệ chia tập train/test/validation đó
def SARIMAX_with_bank_and_ratio(bank, train_ratio, test_ratio, val_ratio,SEASONAL_ARR):
    #in markdown để biết đang chạy ngân hàng nào và tỉ lệ chia tập train/test/validation nào
    display(Markdown('## '+bank+' '+str(train_ratio*10)[0]+'-'+str(test_ratio*10)[0]+'-'+str(val_ratio*10)[0]))

    #Đọc dữ liệu từ file csv
    df=data_preprocessing(bank)
    vni_df = data_preprocessing('VN Index Historical Data.csv')

    #Tính số lượng dữ liệu train/test/validation
    train_size,test_size,val_size=get_split_size(df, train_ratio, test_ratio)

    #Chia tập dữ liệu thành 3 tập train/test/validation
    train, test, val = split_data(df,train_size,test_size,val_size)
    vni_train,vni_test,vni_val = split_data(vni_df, train_size,test_size,val_size)

    #Tạo model
    model = build_model(train,vni_train,SEASONAL_ARR)

    #Tạo tên đồ thị
    title='SARIMAX '+bank[:-4]+' '+str(train_ratio*10)[0]+'-'+str(test_ratio*10)[0]+'-'+str(val_ratio*10)[0]

    #Dự đoán tập test và tập validation
    pred_test,pred_val=predict_test_and_val(model,train_size,test_size,val_size,test,val,vni_test,vni_val)

    #Vẽ đồ thị và in ra độ lệch
    visualize_overview(train,test,val,pred_test,pred_val,title)

    #Tính các chỉ số đánh giá
    print('Test:')
    deviation(test,pred_test)
    print('----------------')
    print('Validation:')
    deviation(val,pred_val)